## cython-examples

- https://www.sympy.org/scipy-2017-codegen-tutorial/notebooks/cython-examples.html

## Writing Cython

In this notebook, we'll take a look at how to implement a simple function using Cython. The operation we'll implement is the first-order diff, which takes in an array of length  n:

and returns the following:

First we'll import everything we'll need and generate some data to work with.

In [2]:
import numpy as np
x = np.random.randn(10000)

Below is a simple implementation using pure Python (no NumPy). The %timeit magic command let's us see how long it takes the function to run on the 10,000-element array defined above.

In [3]:
def py_diff(x):
    n = x.size
    y = np.zeros(n-1)
    for i in range(n-1):
        y[i] = x[i+1] - x[i]
    return y

%timeit py_diff(x)

8.18 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now use the exact same function body but add the %%cython magic at the top of the code cell. How much of a difference does simply pre-compiling make?

In [4]:
%load_ext cython

In [5]:
%%cython
import numpy as np
def cy_diff_naive(x):
    n = x.size
    y = np.zeros(n-1)
    for i in range(n-1):
        y[i] = x[i+1] - x[i]
    return y

In [6]:
%timeit cy_diff_naive(x)

5.68 ms ± 768 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


So it didn't make much of a difference. That's because Cython really shines when you specify data types. We do this by annotating the variables used in the function with cdef <type> .... Let's see how much this improves things.

Note: array types (like for the input arg x) can be declared using the [memoryview](http://docs.cython.org/en/latest/src/userguide/memoryviews.html) syntax double[::1] or using np.ndarray[cnp.float64_t, ndim=1].

In [7]:
%%cython
import numpy as np
def cy_diff(double[::1] x):
    cdef int n = x.size
    cdef double[::1] y = np.zeros(n-1)
    cdef int i
    for i in range(n-1):
        y[i] = x[i+1] - x[i]
    return y

That made a huge difference! There are a couple more things we can do to speed up our diff implementation, including [disabling some safety checks](https://cython.readthedocs.io/en/latest/src/reference/compilation.html?#compiler-directives). The combination of disabling bounds checking (making sure you don't try access an index of an array that doesn't exist) and disabling wraparound (disabling use of negative indices) can really improve things when we are sure neither condition will occur. Let's try that.

In [8]:
%%cython
from cython import wraparound, boundscheck
import numpy as np

@boundscheck(False)
@wraparound(False)
def cy_diff2(double[::1] x):
    cdef int n = x.size
    cdef double[::1] y = np.zeros(n-1)
    cdef int i
    for i in range(n-1):
        y[i] = x[i+1] - x[i]
    return y

In [9]:
%timeit cy_diff2(x)

17 μs ± 1.99 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Finally, let's see how NumPy's diff performs for comparison.

In [11]:
def np_diff(x):
    return np.diff(x)

%timeit np_diff(x)

15.2 μs ± 1.05 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Why is NumPy's diff implementation faster? Maybe use the --annotate flag to peek at the C code generated by our latest Cython implementation.